In [2]:
import pyspark
from pyspark.sql import SparkSession

In [3]:
spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

24/03/03 10:24:03 WARN Utils: Your hostname, ubuntu-focal resolves to a loopback address: 127.0.1.1; using 10.0.2.15 instead (on interface enp0s3)
24/03/03 10:24:03 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


24/03/03 10:24:05 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/03/03 10:24:07 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [4]:
spark.version

'3.3.2'

In [5]:
import pandas as pd
from pyspark.sql import types

In [6]:
df = pd.read_csv('fhv_tripdata_2019-10.csv')

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1897493 entries, 0 to 1897492
Data columns (total 7 columns):
 #   Column                  Dtype  
---  ------                  -----  
 0   dispatching_base_num    object 
 1   pickup_datetime         object 
 2   dropOff_datetime        object 
 3   PUlocationID            float64
 4   DOlocationID            float64
 5   SR_Flag                 float64
 6   Affiliated_base_number  object 
dtypes: float64(3), object(4)
memory usage: 101.3+ MB


In [8]:
schema = types.StructType([
    types.StructField('dispatching_base_num', types.StringType(), True),
    types.StructField('pickup_datetime', types.TimestampType(), True),
    types.StructField('dropOff_datetime', types.TimestampType(), True),
    types.StructField('PULocationID', types.IntegerType(), True),
    types.StructField('DOLocationID', types.IntegerType(), True),
    types.StructField('SR_Flag', types.StringType(), True)
])

In [9]:
df = spark.read \
    .option("header", "true") \
    .schema(schema) \
    .csv('fhv_tripdata_2019-10.csv')

In [10]:
df = df.repartition(6)

In [11]:
df.write.parquet('fhv/2019/10/', mode='overwrite')

24/03/03 10:24:57 WARN CSVHeaderChecker: Number of column in CSV header is not equal to number of fields in the schema:
 Header length: 7, schema size: 6
CSV file: file:///home/vagrant/notebooks/fhv_tripdata_2019-10.csv


In [12]:
df = spark.read.parquet('fhv/2019/10/')

In [13]:
df.printSchema()

root
 |-- dispatching_base_num: string (nullable = true)
 |-- pickup_datetime: timestamp (nullable = true)
 |-- dropOff_datetime: timestamp (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- SR_Flag: string (nullable = true)



In [14]:
from pyspark.sql import functions as F

In [15]:
df \
    .withColumn('pickup_date', F.to_date(df.pickup_datetime)) \
    .withColumn('dropOff_date', F.to_date(df.dropOff_datetime)) \
    .select('pickup_date', 'dropOff_date', 'PULocationID', 'DOLocationID') \
    .filter(F.to_date(df.pickup_datetime) == '2019-10-15') \
    .count() 

62610

In [16]:
df.registerTempTable('fhv_trips_data')

/opt/spark/python/pyspark/sql/dataframe.py:229: FutureWarning: Deprecated in 2.0, use createOrReplaceTempView instead.
  warnings.warn("Deprecated in 2.0, use createOrReplaceTempView instead.", FutureWarning)


In [21]:
spark.sql("""
SELECT
    pickup_datetime,
    dropOff_datetime,
    TIMESTAMPDIFF(HOUR, pickup_datetime, dropOff_datetime) as trip_length
FROM
    fhv_trips_data
order by trip_length desc

""").show()

+-------------------+-------------------+-----------+
|    pickup_datetime|   dropOff_datetime|trip_length|
+-------------------+-------------------+-----------+
|2019-10-11 18:00:00|2091-10-11 18:30:00|     631152|
|2019-10-28 09:00:00|2091-10-28 09:30:00|     631152|
|2019-10-31 23:46:33|2029-11-01 00:13:00|      87672|
|2019-10-01 21:43:42|2027-10-01 21:45:23|      70128|
|2019-10-17 14:00:00|2020-10-18 00:00:00|       8794|
|2019-10-26 21:26:00|2020-10-26 21:36:00|       8784|
|2019-10-30 12:30:04|2019-12-30 13:02:08|       1464|
|2019-10-25 07:04:57|2019-12-08 07:54:33|       1056|
|2019-10-25 07:04:57|2019-12-08 07:21:11|       1056|
|2019-10-01 13:41:00|2019-11-03 14:58:51|        793|
|2019-10-01 13:47:17|2019-11-03 15:20:28|        793|
|2019-10-01 07:21:12|2019-11-03 08:44:21|        793|
|2019-10-01 06:11:45|2019-11-03 06:39:03|        792|
|2019-10-01 05:36:30|2019-11-03 06:23:36|        792|
|2019-10-01 19:09:17|2019-11-03 19:41:02|        792|
|2019-10-01 11:59:55|2019-11

In [24]:
df_zones = spark.read \
    .option("header", "true") \
    .csv('taxi_zone_lookup.csv')

In [26]:
df_zones.registerTempTable('zones')

In [27]:
spark.sql("""
select * from zones
""").show()

+----------+-------------+--------------------+------------+
|LocationID|      Borough|                Zone|service_zone|
+----------+-------------+--------------------+------------+
|         1|          EWR|      Newark Airport|         EWR|
|         2|       Queens|         Jamaica Bay|   Boro Zone|
|         3|        Bronx|Allerton/Pelham G...|   Boro Zone|
|         4|    Manhattan|       Alphabet City| Yellow Zone|
|         5|Staten Island|       Arden Heights|   Boro Zone|
|         6|Staten Island|Arrochar/Fort Wad...|   Boro Zone|
|         7|       Queens|             Astoria|   Boro Zone|
|         8|       Queens|        Astoria Park|   Boro Zone|
|         9|       Queens|          Auburndale|   Boro Zone|
|        10|       Queens|        Baisley Park|   Boro Zone|
|        11|     Brooklyn|          Bath Beach|   Boro Zone|
|        12|    Manhattan|        Battery Park| Yellow Zone|
|        13|    Manhattan|   Battery Park City| Yellow Zone|
|        14|     Brookly

In [34]:
spark.sql("""
select
    PULocationID as pickup_zone_id,
    Zone,
    count(1) as pickup_frequency
from
    fhv_trips_data
left join zones on PULocationID = LocationID
group by pickup_zone_id, Zone
order by pickup_frequency asc 
""").show()

+--------------+--------------------+----------------+
|pickup_zone_id|                Zone|pickup_frequency|
+--------------+--------------------+----------------+
|             2|         Jamaica Bay|               1|
|           105|Governor's Island...|               2|
|           111| Green-Wood Cemetery|               5|
|            30|       Broad Channel|               8|
|           120|     Highbridge Park|              14|
|            12|        Battery Park|              15|
|           207|Saint Michaels Ce...|              23|
|            27|Breezy Point/Fort...|              25|
|           154|Marine Park/Floyd...|              26|
|             8|        Astoria Park|              29|
|           128|    Inwood Hill Park|              39|
|           253|       Willets Point|              47|
|            96|Forest Park/Highl...|              53|
|            34|  Brooklyn Navy Yard|              57|
|            59|        Crotona Park|              62|
|         